In [1]:
import os

import torch

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from PIL import Image
from torchvision import transforms
from d2l import torch as d2l
d2l.set_figsize()

In [ ]:
HR_img_path = os.path.join('..', 'learn_gan', 'CycleGan', 'datasets', 'train')
LR_img_path = os.path.join('.', 'LR')

img = d2l.Image.open(HR_img_path + os.path.join('\\2013-11-08 16_45_24.jpg'))
d2l.plt.imshow(img)

In [ ]:
lr_img = transforms.Compose([
    transforms.Resize(128 // 2)
])(img)
d2l.plt.imshow(lr_img)

In [ ]:
hr_img = transforms.Compose([
    transforms.CenterCrop(128)
])(img)
d2l.plt.imshow(hr_img)

In [ ]:
import glob
origin_img_list = glob.glob('../learn_gan/CycleGan/datasets/train/*.*')
origin_img_list

In [ ]:
idx = 0
for img_path in origin_img_list:
    hr_img = Image.open(img_path)
    lr_img = transforms.Compose([transforms.Resize(128 // 2)])(hr_img)
    hr_img.save('HR/' + str(idx) + '.jpg')
    lr_img.save('LR/' + str(idx) + '.jpg')
    idx += 1

In [ ]:
import glob
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms

class ImgLoader(Dataset):
    def __init__(self, root_path='../super_resolution'):
        self.transformer = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
            )
        ])
        self.lr_list = sorted(glob.glob(root_path + '/LR/*.*'))
        self.hr_list = sorted(glob.glob(root_path + '/HR/*.*'))

    def __len__(self):
        return min(len(self.lr_list), len(self.hr_list))

    def __getitem__(self, index):
        return {
            'lr_img_path': self.lr_list[index],
            'hr_img_path': self.hr_list[index]
        }


loader = DataLoader(ImgLoader(),
                    batch_size=4, shuffle=True, drop_last=True)
for i, dicts in enumerate(loader):
    print("===============================")
    print('lr_path = ')
    print(dicts['lr_img_path'])
    print('hr_path = ')
    print(dicts['hr_img_path'])
    print("===============================")

In [2]:
import model_g
import torch
import torchvision.transforms
img_hr_path = 'E:\毕设\style_transfer\\res_img\\res2023_03_15_14_32_49.jpg'
hr_img = Image.open(img_hr_path)
hr_img.show()

In [3]:
device = torch.device('cuda:0')
generator = model_g.Generator()
param_dict = torch.load('./output/dicts_p.pt')
generator.load_state_dict(param_dict['generator'])
generator.to(device)
generator.eval()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
lr_img = transforms.Compose([
    transforms.Resize(128)
])(hr_img)
lr_img.show()

In [7]:
img = transform(lr_img).to(device)
pre = generator(img[None, ...])[0]

def UnNormalize(img, mean=None, std=None):
        if mean is None:
            mean = [0.485, 0.456, 0.406]
        if std is None:
            std = [0.229, 0.224, 0.225]
        for i in range(3):
            img[i] = std[i]*img[i] + mean[i]
        return img

pre = UnNormalize(pre).cpu().clamp(0.0, 1.0)
sr_img = transforms.ToPILImage()(pre)
sr_img.show()